In [1]:
from models import Ensemble, Gru
import pandas as pd
import tensorflow as tf
import numpy as np
import os
from dataset import LABELS

# Prepare data

In [3]:
# If you download the file from https://drive.google.com/file/d/1DVdjHTMePQvo_fouEO17QIcfPPstDpmm/view?usp=sharing save features_test.pkl as test data and silence_detected.pkl as silence.

test_data = pd.read_pickle('extracted_features\\features_test.pkl')
test = tf.data.Dataset.from_tensor_slices((test_data, np.zeros(len(test_data))))

silence = pd.read_pickle('extracted_features\\silence_detected.pkl')

In [5]:
LABELS_dict = {i:LABELS[i] for i in range(len(LABELS))}
LABELS_dict[10] = 'unknown'

# Single, best GRU model submission

In [ ]:
pred = Gru(from_path='models\\best_gru_label_vs_unknown.h5').predict(test)

In [ ]:
x = pd.concat([pd.DataFrame(pred), pd.DataFrame(silence)], axis=1)
x.columns = ['label', 'silence']
res = np.where(x['silence'] == 0 , 'silence', x['label'].apply(lambda x: LABELS_dict[x]))
res = pd.DataFrame(res)
res.columns = ['label_prediction']

In [ ]:
df = pd.read_csv('submissions\\sample_submission.csv', header = 0)
df['label'] = res
df.to_csv(f'submissions\\submission_best_gru.csv', index=False, header=True)

# Ensemble submission

In [3]:
model_paths = os.listdir('models\\ensemble\\')
model_paths = ['models\\ensemble\\' + path for path in model_paths]

In [4]:
ensemble = Ensemble(model_paths=model_paths)

## Prepare submission mean

In [ ]:
pred = ensemble.predict_mean(test.batch(10))

In [7]:
x = pd.concat([pd.DataFrame(pred), pd.DataFrame(silence)], axis=1)
x.columns = ['label', 'silence']
res = np.where(x['silence'] == 0 , 'silence', x['label'].apply(lambda x: LABELS_dict[x]))
res = pd.DataFrame(res)
res.columns = ['label_prediction']

In [8]:
df = pd.read_csv('submissions\\sample_submission.csv', header = 0)
df['label'] = res
df.to_csv(f'submissions\\submission_ensemble_mean.csv', index=False, header=True)

## Prepare submission max

In [ ]:
pred2 = ensemble.predict_max(test.batch(10))

In [ ]:
x = pd.concat([pd.DataFrame(pred2), pd.DataFrame(silence)], axis=1)
x.columns = ['label', 'silence']
res = np.where(x['silence'] == 0 , 'silence', x['label'].apply(lambda x: LABELS_dict[x]))
res = pd.DataFrame(res)
res.columns = ['label_prediction']

In [ ]:
df = pd.read_csv('submissions\\sample_submission.csv', header = 0)
df['label'] = res
df.to_csv(f'submissions\\submission_ensemble_max.csv', index=False, header=True)